# Basic Topic Modelling with a Latent Dirichlet Allocation (LDA) Visualisation

---
---

## Introduction to Latent Dirichlet Allocation (LDA)
If you've heard of topic modelling before, you may have heard of Latent Dirichlet Allocation. LDA is a popular statistical model for topics and one that is almost synonymous with topic modelling in general.

It's important to understand that LDA is only _one_ type of topic model; there are many others with equally dull acronyms (e.g. LSA, HDP, LSI, NNMF). Also, Gensim provides an _implementation_ of LDA called `LdaModel` (based on the LDA mathematics), but there are many other implementations in different libraries and software. Different implementations of LDA should give you more or less the same results, but different topic models may well give you different results.

---
### Going Further: More About LDA and Topic Modelling

The mathematics behind LDA is out of the scope of this workshop. Likewise, the other types of topic model are too much for us to cover in this beginners' class. However, if you wish to go further there are links to extension material in the last section of this notebook 'What's Next?'.

---

So, topic modelling with LDA. **Let's dive right in! 🌊**

We have a corpus of 56 presidential speeches that we've prepared into nicely cleaned tokens.

> Our question: **What are the underlying themes of these texts as a group?**

---
---


## Loading the Corpus into Gensim from Files

First, we need to get a list of all the files in the `data/inaugural` folder:

In [1]:
from pathlib import Path
inaugural = Path('data', 'inaugural')
files = list(inaugural.iterdir())

Let's open all the files in turn and add their contents to a big list of strings call `text`:

In [2]:
text = []
for file in files:
    with open(file, 'r') as reader:       
        document = []
        for token in reader.read().split():
            document.append(token)
        text.append(document)

print(f'There are {len(text)} documents loaded.')

There are 56 documents loaded.


(👆👆👆 If you don't understand this code above yet, don't worry. You can skip over it and still follow along with the topic modelling.)

### Loading the Tokens into a Dictionary

Now we have to load the corpus from text files into a _dictionary_. Gensim provides a special class of dictionary for us to work with called `gensim.corpora.Dictionary`. It has some extra stuff in it over and above the ordinary Python `dict`, but we don't need to worry about the details.

In [3]:
from gensim.corpora import Dictionary

dictionary = Dictionary(text)
str(dictionary)

"Dictionary(6164 unique tokens: ['5', 'accomplish', 'accordingly', 'acknowledge', 'acquit']...)"

From this we can understand that Gensim has found 6164 unique tokens in the corpus. But exactly what information does this `Dictionary` contain?

In short, it is a _mapping_ between each _token_ and a _unique id number_:

In [4]:
dictionary.token2id

{'5': 0,
 'accomplish': 1,
 'accordingly': 2,
 'acknowledge': 3,
 'acquit': 4,
 'act': 5,
 'actual': 6,
 'actuate': 7,
 'add': 8,
 'addition': 9,
 'address': 10,
 'administration': 11,
 'adopt': 12,
 'adore': 13,
 'adorn': 14,
 'advance': 15,
 'advancement': 16,
 'advantage': 17,
 'advantageously': 18,
 'affair': 19,
 'affect': 20,
 'affection': 21,
 'affectionate': 22,
 'agency': 23,
 'aid': 24,
 'all': 25,
 'allot': 26,
 'almighty': 27,
 'alteration': 28,
 'american': 29,
 'among': 30,
 'animosity': 31,
 'anticipation': 32,
 'anxiety': 33,
 'appreciation': 34,
 'ardent': 35,
 'arduous': 36,
 'arise': 37,
 'article': 38,
 'assemblage': 39,
 'assemble': 40,
 'assure': 41,
 'asylum': 42,
 'attachment': 43,
 'attention': 44,
 'attribute': 45,
 'auspiciously': 46,
 'author': 47,
 'aver': 48,
 'avoid': 49,
 'await': 50,
 'awaken': 51,
 'be': 52,
 'behold': 53,
 'benediction': 54,
 'benefit': 55,
 'benign': 56,
 'besides': 57,
 'bind': 58,
 'birth': 59,
 'bless': 60,
 'blessing': 61,
 'brie

### Vocabulary Size and Filtering Extremes

The tokens are collectively known as a _vocabulary_ and the size and nature of the vocabulary you choose is important. A large vocabulary will take more computational power and memory to analyse. A vocabulary with many rare words has less useful information in it (so you are wasting time and memory processing it).

To reduce the size of the vocabulary and increase the density of information content, we can filter out the extremes with `filter_extremes()`. You can experiment with different values, but here we filter out tokens that appear less than 5 times or appear in more than 50% of the documents:

In [5]:
dictionary.filter_extremes(no_below=5, no_above=0.5)
str(dictionary)

"Dictionary(1679 unique tokens: ['accomplish', 'acknowledge', 'actual', 'add', 'addition']...)"

Now we have 1679 unique tokens, compared with 6164 for the original `Dictionary`.

### Saving the Dictionary To File

Gensim provides an easy way to save the `Dictionary` to file so you can reload it later.

In [6]:
dict_file = str(Path('data', 'saved', '1-dictionary.gensim')) # Transform Path to string as `save()` only accepts strings
dictionary.save(dict_file)

You can check that we now have a file named `1-dictionary.gensim` in the `data/saved` folder. NB: This file is not human-readable.

### Loading the Dictionary From File

Here is how you can load the dictionary, or any other object you create in Gensim, back into the notebook:

In [7]:
dictionary = Dictionary.load(dict_file)
str(dictionary)

"Dictionary(1679 unique tokens: ['accomplish', 'acknowledge', 'actual', 'add', 'addition']...)"

Once we have the corpus loaded we can start to analyse its contents. The first step is to count the words to create a **bag-of-words** corpus.

---
---
## Bag of Words Corpus

>**The key to understanding Natural Language Processing (NLP) is that the computer can only do computations on _numbers_. We have to present our corpus for analysis in a numerical form — typically _vectors_ — and make human sense of everything at the end.**

A bag-of-words (BoW) corpus is the _vocabulary_ of known tokens (words) in the corpus together with some _measure_ of how often they occur. The measurement may be:
* binary (presence or absence)
* count (how many times the token occurs)
* frequency (count divided by the total number of tokens).

In our example, we will use Gensim's `doc2bow()`, which simply counts the tokens:

In [8]:
corpus = [dictionary.doc2bow(doc) for doc in text]
corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 2),
 (21, 1),
 (22, 2),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 2),
 (38, 1),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 3),
 (43, 1),
 (44, 1),
 (45, 1),
 (46, 1),
 (47, 2),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 1),
 (55, 1),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 1),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 2),
 (65, 1),
 (66, 2),
 (67, 1),
 (68, 2),
 (69, 1),
 (70, 1),
 (71, 1),
 (72, 1),
 (73, 1),
 (74, 1),
 (75, 2),
 (76, 1),
 (77, 2),
 (78, 1),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 1),
 (86, 1),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 1),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 1),
 (96, 1),
 (97, 1),
 (98, 3),
 (99, 1),
 (100, 1),

What `corpus[0]` shows us is a list of _token ids_ and their _count_ for the first document in the corpus. For example, `(7, 1)` is the token id `7` and its count `1` i.e. it was found once in this document. 

You can look at the counts for any of the documents from 0-55 by changing the index number (remember indexing starts at 0).

### The 'Bag' in Bag of Words

In this basic BoW model the order and location of words is discarded. For example, it does not matter if the words 'red' and 'nose' are adjacent ('red nose'), or at the beginning or end of a sentence; BoW just treats the words individually. It is like a 'bag' of Scrabble™ tiles, where each tile is a word, all rattling around together in no particular order.

---
## Going Further: Term Frequency–Inverse Document Frequency (TF-IDF)
Highly frequent words can come to dominate your results and yet they may not be as meaningful or interesting as rarer words. For example, if you are looking at articles about the history of the Moon landings, even if you have removed all the stopwords, you may well find that the words 'lunar', 'moon', 'landing', 'orbit', and 'earth' predominate. Subtle differences in topic between documents may be lost.

We have already done some filtering of extremes (above) by filtering tokens that appeared less than 5 times or appeared in more than 50% of the documents.

Another way to deal with this is to use a _weighting factor_ called **TF-IDF**. A value is calculated for each word that reflects:
* Term frequency (TF) - the number of times the word appears in the document
* Document frequency (DF) - the number of documents in the corpus that contain the word

For example, if a very uncommon word is present in two documents, this word is weighted more highly than a word that is present in all documents in a corpus.

Gensim provides `TfidfModel`, which accepts a BoW corpus and creates the weightings for each token in each document:

In [9]:
from gensim.models.tfidfmodel import TfidfModel
tfidf = TfidfModel(corpus)
tfidf.num_docs

56

We can take a peak at the weightings for any document like this, where the first number in each pair (tuple) is the _token id_ and the second is the _weighting_:

In [10]:
for doc in tfidf[corpus[0]]:
    print(doc)

(0, 0.03371549584954543)
(1, 0.04539502752050589)
(2, 0.06371997697331631)
(3, 0.029138908511114364)
(4, 0.07911052337005132)
(5, 0.05044278256396589)
(6, 0.03716558815461547)
(7, 0.05641300960979839)
(8, 0.03211783311115547)
(9, 0.026408528486027517)
(10, 0.04102246321306336)
(11, 0.05329202148740097)
(12, 0.05641300960979839)
(13, 0.03211783311115547)
(14, 0.027745268803712947)
(15, 0.07314029632421884)
(16, 0.04539502752050589)
(17, 0.059863101914868426)
(18, 0.059863101914868426)
(19, 0.03903727417027296)
(20, 0.11282601921959678)
(21, 0.05641300960979839)
(22, 0.07079025013026456)
(23, 0.06809254128075884)
(24, 0.03716558815461547)
(25, 0.05641300960979839)
(26, 0.04313581520044797)
(27, 0.05044278256396589)
(28, 0.05329202148740097)
(29, 0.07911052337005132)
(30, 0.030594507727148022)
(31, 0.06371997697331631)
(32, 0.023888393745265055)
(33, 0.047821736832581174)
(34, 0.023888393745265055)
(35, 0.03371549584954543)
(36, 0.06371997697331631)
(37, 0.050248446144656456)
(38, 0.07314

This model could be used instead of the BoW model as input to topic modelling algorithms.

---

---
---


## Training the Latent Dirichlet Allocation (LDA) Model

### What Does 'Training a Model' Mean?

The first thing we need to do is 'train a model'. These words and concepts come from the (fashionable!) sphere of **machine learning**. A 'model' is a simplified representation of something in the real world that a computer can handle. In topic modelling, the model is a representation of the topics of the corpus.

We can say the computer acquires 'experience', or a simplified 'understanding' of what the corpus is about by creating a model of it. We have to train the computer in how to make its model by feeding it training data -- in the case of topic modelling, this data is the text we are interested in.

Typically, in a machine learning project you would split your data into a _training set_ and a _holdout set_. The training set is used to train the model, and the holdout set is used to test the model to see if it is valid or useful.

As we are doing a simple exploratory analysis here we are going to use all the data at once. We won't be doing any validation in this workshop.

**Let's get started with our topic modelling!**

We `import` the `LdaModel`, pass it our BoW `corpus` and limit the number of topics we are interested in to `5`. Feel free to experiment with the number of topics.

In [11]:
from gensim.models.ldamodel import LdaModel

ldamodel = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

Then we save the LDA model to file:

In [12]:
lda_file = str(Path('data', 'saved', '2-lda.gensim'))
ldamodel.save(lda_file)

### Viewing Topics for the Whole Corpus

> **❗Important Note: You may see the topics, words and probabilities created differently than the examples below due to the way they are generated. This means my descriptions may not match up exactly with what you see when you run the notebook.**

To view the topics we can use `show_topics` — and we can optionally limit it to the number of topics and words we are interested in:

In [13]:
ldamodel.show_topics(num_topics=5, num_words=5)

[(0,
  '0.007*"congress" + 0.006*"executive" + 0.004*"general" + 0.004*"ought" + 0.004*"business"'),
 (1,
  '0.007*"institution" + 0.006*"opinion" + 0.005*"revenue" + 0.005*"object" + 0.004*"authority"'),
 (2,
  '0.016*"there" + 0.012*"friend" + 0.009*"word" + 0.009*"today" + 0.007*"child"'),
 (3,
  '0.010*"case" + 0.006*"democracy" + 0.006*"program" + 0.006*"minority" + 0.005*"provision"'),
 (4,
  '0.008*"today" + 0.007*"help" + 0.007*"century" + 0.007*"americans" + 0.005*"opportunity"')]

What can we see here? These 5 topics represent the topic distribution of the **corpus as a whole**.

Let's take an example topic:

```
(3, '0.008*"congress" + 0.007*"business" + 0.005*"increase" + 0.004*"trade" + 0.004*"ought"')
```

The first number `3` in the tuple is the topic number. In front of each word is the probability of that word making up the topic. For example, `0.007*"business"` means that the topic is 0.7% business-y.

Overall, the topic _appears_ to be about the role of congress in increasing business and trade.

### Topics for an Individual Document
We can ask the model to give us the topic distribution for any individual document.

For example, if we pass in document `10` from the BoW corpus, it gives us two topics for that document: it is 56% topic `0` and 44% topic `3`.

In [14]:
ldamodel.get_document_topics(corpus[10])

[(0, 0.013316183), (1, 0.9844907)]

## Visualising Topics with pyLDAvis
Understanding the data that underlies a topic model is vital, but fortunately we also have a slightly more human-friendly option to help us interpret the topics!

[pyLDAvis](https://github.com/bmabey/pyLDAvis) is a library for creating interactive topic model visualisations. It even has a helper function specifically for Gensim that we can use.

> **It will take a while to load this visualisation!**

In [15]:
# Silence an annoying warning we cannot do anything about
import warnings
warnings.filterwarnings('ignore')

# pyLDAvis code starts here
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/mary/digitalhumanities/intro-to-text-mining-with-python/venv/lib/python3.7/site-packages/past/types/oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.087913 -0.021614       1        1  47.815632
4     -0.113353  0.005162       2        1  31.704443
1      0.119048 -0.033252       3        1  15.354429
3      0.000495  0.117563       4        1   3.585795
2     -0.094103 -0.067858       5        1   1.539707, topic_info=     Category        Freq            Term       Total  loglift  logprob
494   Default   74.000000           there   74.000000  30.0000  30.0000
1578  Default   83.000000           today   83.000000  29.0000  29.0000
1564  Default   69.000000            help   69.000000  28.0000  28.0000
1207  Default   53.000000       democracy   53.000000  27.0000  27.0000
369   Default   43.000000          friend   43.000000  26.0000  26.0000
694   Default   55.000000           child   55.000000  25.0000  25.0000
842   Default   39.000000            word   39.000000  24.0000  24.0000
407   Default   54.000000            like   54.000000  23.0000  23.0000
578   Default   60.000000      generation   60.000000  22.0000  22.0000
1078  Default   41.000000            turn   41.000000  21.0000  21.0000
1620  Default   62.000000       americans   62.000000  20.0000  20.0000
315   Default  126.000000        congress  126.000000  19.0000  19.0000
302   Default   36.000000            case   36.000000  18.0000  18.0000
359   Default   34.000000            fact   34.000000  17.0000  17.0000
1306  Default   68.000000         century   68.000000  16.0000  16.0000
104   Default   95.000000       executive   95.000000  15.0000  15.0000
336   Default   45.000000      difference   45.000000  14.0000  14.0000
1633  Default   28.000000           woman   28.000000  13.0000  13.0000
1023  Default   52.000000            make   52.000000  12.0000  12.0000
886   Default   33.000000        remember   33.000000  11.0000  11.0000
860   Default   46.000000            face   46.000000  10.0000  10.0000
1495  Default   32.000000       challenge   32.000000   9.0000   9.0000
166   Default   64.000000          object   64.000000   8.0000   8.0000
34    Default   38.000000           bless   38.000000   7.0000   7.0000
605   Default   36.000000          moment   36.000000   6.0000   6.0000
1508  Default   43.000000         problem   43.000000   5.0000   5.0000
1609  Default   43.000000        economic   43.000000   4.0000   4.0000
294   Default   66.000000       authority   66.000000   3.0000   3.0000
59    Default   50.000000  constitutional   50.000000   2.0000   2.0000
1598  Default   26.000000           unity   26.000000   1.0000   1.0000
...       ...         ...             ...         ...      ...      ...
1096   Topic5    0.829715        attitude    7.220186   2.0100  -6.2832
1163   Topic5    0.675646            send    5.968588   1.9950  -6.4886
1484   Topic5    2.361961           young   20.931826   1.9918  -5.2370
392    Topic5    0.676258    intellectual    6.059323   1.9808  -6.4877
949    Topic5    0.675603           throw    6.079240   1.9765  -6.4886
842    Topic5    4.089443            word   39.019928   1.9179  -4.6881
494    Topic5    6.978973           there   74.391724   1.8071  -4.1536
359    Topic5    2.990535            fact   34.445591   1.7296  -5.0010
1587   Topic5    1.799534            hard   19.592075   1.7860  -5.5090
1633   Topic5    2.362183           woman   28.557753   1.6812  -5.2369
1492   Topic5    1.860101      washington   20.885019   1.7552  -5.4759
1078   Topic5    2.974324            turn   41.167377   1.5459  -5.0065
886    Topic5    2.372295        remember   33.237762   1.5337  -5.2326
1578   Topic5    4.088375           today   83.722397   1.1542  -4.6883
786    Topic5    1.842057           offer   25.322342   1.5528  -5.4856
694    Topic5    3.028336           child   55.753880   1.2606  -4.9885
407    Topic5    2.990397            like   54.731598   1.2665  -5.0011
1207   Topic5    2.943614       democracy   53.8510

Here are some hints to help you interpret the visualisation:

* On the **left-hand side** is a scatterplot of some bubbles:
 * Each **bubble** represents a topic.
 * The **size of a bubble** represents how _prevalent_ or popular the topic is overall.
 * The **distance** from one bubble to another represents how similar the topics are to each other. If they overlap then the topics share significant similarity.
 
* On the **right-hand side** is a histogram of terms (tokens):
 * Select a bubble and it shows the top-30 **most relevant terms** for that topic.
 * The **red bar** represents how frequent a term is in the topic.
 * The **blue bar** represents how frequent the term is overall in all topics. So a long red bar with only a short blue bar indicates a term that is highly specific to that particular topic. Conversely, a red bar with a long blue bar means the term is also present in many other topics.
 * By mousing over a particular term, the size of the bubbles changes to show the relative frequency of that term in the various topics.
 * By adjusting the slide, it adjusts the **_relevance_ value (λ)**, which is the weight given to whether a term appears exclusively in a particular topic or is spread over topics more evenly. If λ = 1 terms are ranked according to their probabilities in the particular topic only; if λ = 0 terms are ranked higher if they are unusual terms that occur almost exclusively in that topic. Typically, the optimal value is around 0.6, but it is interesting to adjust it and observe any differences.

---
---
## Summary

Well done for getting to the end of the topic modelling fundamentals notebook! Here is what we have done:

* Understood that Latent Dirichlet Allocation (LDA) is one type of topic modelling
* Loaded and saved the cleaned tokens into a Gensim corpus dictionary
* Created a bag-of-words (BoW) corpus
* Looked at Term Frequency–Inverse Document Frequency (TF-IDF)
* Trained a Latent Dirichlet Allocation (LDA) topic model
* Visualised the resulting topics with pyLDAvis


👌👌👌

Congratulations! 🎉

---
---
## What's Next?
If you have decided that text-mining with Python might be for you, then here are some more resources to study in your own time.

### Topic Modelling

* For a general overview of the issues that should concern you as a humanist: [Topic Modeling for Humanists: A Guided Tour](http://scottbot.net/topic-modeling-for-humanists-a-guided-tour/).
* Follow a more advanced Jupyter notebook: [Topic Modelling - and more - with Gensim!](https://github.com/bhargavvader/personal/blob/master/notebooks/text_analysis_tutorial/topic_modelling.ipynb).

### Text-mining in General

* Follow a more in-depth set of Jupyter notebooks with [The Art of Literary Text Analysis](https://github.com/sgsinclair/alta/blob/master/ipynb/ArtOfLiteraryTextAnalysis.ipynb).
* Read a practical and well-explained approach to Natural Language Processing (NLP) in Python: Srinivasa-Desikan, B., 2018. _Natural Language Processing and Computational Linguistics : A practical guide to text analysis with Python, Gensim, spaCy, and Keras._ Birmingham: Packt Publishing. [Available online](https://idiscover.lib.cam.ac.uk/primo-explore/fulldisplay?docid=44CAM_NPLD_MARC018975982&context=L&vid=44CAM_PROD&search_scope=SCOP_CAM_ALL&tab=cam_lib_coll&lang=en_US). This book has chapters on text pre-processing steps, topic modelling, Gensim - and comes with Jupyter notebooks to follow.

### Learning Python and Jupyter Notebooks for Digital Humanities:

* I have written a guide to running these notebooks on your own computer in the [README](https://github.com/mchesterkadwell/intro-to-text-mining-with-python/blob/master/README.md) of the GitHub code repository.
* Install Python using Anaconda on your computer: [Installing Anaconda on Windows](https://www.datacamp.com/community/tutorials/installing-anaconda-windows) [Installing Anaconda on Mac](https://www.datacamp.com/community/tutorials/installing-anaconda-mac-os-x).
* Learn how to run and write Jupyter notebooks on your own computer (rather than using Binder): [Jupyter Notebook Tutorial: The Definitive Guide](https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook).
* Work through Chapters 1 - 4 (online Jupyter notebooks) of [Python Programming for the Humanities](http://www.karsdorp.io/python-course/).
* Browse a big list of resources for [Teaching Yourself to Code in DH](http://scottbot.net/teaching-yourself-to-code-in-dh/).

Even if you are not sure programming is for you, [Cambridge Digital Humanities](https://www.cdh.cam.ac.uk/) (CDH) has a number of resources to support your research. 

* CDH Learning - [training events/workshops](https://www.cdh.cam.ac.uk/learning/cdh-events) and mentoring programme.
* CDH Lab - email [lab@cdh.cam.ac.uk](mailto:lab@cdh.cam.ac.uk) for advice on your project, whether you are just getting started, somewhere in the middle, or thinking about the future.